In [1]:
'''
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install tensorflow
!pip install keras
!pip install matplotlib
!pip install seaborn
!pip install plotly
!pip install tqdm
'''

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential

from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

### Configuring TPU's as we will be using Bert 

In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [22]:
## From : https://stackoverflow.com/questions/40305122/log-file-to-pandas-dataframe
import re
import pandas as pd
def logFile2DataFrame( filePath ) :
    dfLog = pd.DataFrame( columns=[ 'Date','Time','Pid','Level','Component','Content' ] )
    
    with open( filePath , 'r' ) as logFile :
        numRows = -1
        for line in logFile :
            tokens    = line.split(" ")
            Date = tokens[0]
            Time    = tokens[1]
            Pid = tokens[2]
            Level     = tokens[3] 
            Component = tokens[4] 
            Content   = " ".join( tokens[5:] )
            numRows += 1
            dfLog.loc[ numRows ] = [ Date , Time , Pid , Level , Component , Content ]

    return dfLog

In [18]:
def Load_HDFS_Data(logfile):
    df = pd.read_csv(logfile, engine='c', delimiter=" ", na_filter=False, memory_map=True, dtype={'Date':object, "Time": object, "Pid": object , "Level": object , "Component": object , "Content": object})
    return df

In [23]:

train = logFile2DataFrame('../Datasets/HDFS/HDFS.log')
#validation = pd.read_csv('../Datasets/Sentiment/validation.csv')
#test = pd.read_csv('../Datasets/Sentiment/test.csv')

In [26]:
train


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,1,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,1,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",1,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",1,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",1,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,1,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,1,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [28]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)


In [29]:
train.head()

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [30]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

### Check the maximum number of words in a comment

In [31]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

In [32]:
#Writing a function for getting auc score for validation

def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [33]:
# Data Preparation

xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, 
                                                  stratify=train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

### Simple RNN

In [42]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [44]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         13049100  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 13,089,301
Trainable params: 13,089,301
Non-trainable params: 0
_________________________________________________________________
CPU times: total: 406 ms
Wall time: 166 ms


In [46]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/5
 47/150 [========>.....................] - ETA: 2:13 - loss: 0.3299 - accuracy: 0.8963